In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/lin

In [87]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [100]:
df1=spark.read.option("multiline","true").json('/content/1.json')
df1.show()

+---+
|  A|
+---+
|[1]|
+---+



In [101]:
df2=spark.read.option("multiline","true").json('/content/2.json')
df2.show()

+---+---+
|  A|  C|
+---+---+
|[1]|  2|
+---+---+



In [102]:
df3=spark.read.option("multiline","true").json('/content/3.json')
df3.show()

+------+---+
|     A|  C|
+------+---+
|[1, 3]|  2|
+------+---+



In [104]:
def all_columns_apart_from_the_particular_column(df,name_string):
  ms=df.schema.fields;
  ag=[];
  for i in ms:
    if(i.name!=name_string):
      ag.append(i.name);
  return ag;

In [105]:
def expand_structure(dfstruct,al):
  ag=[];
  for i in dfstruct:
    ag.append(al+"."+i.name);
  return ag[::-1];

In [112]:
def flat_df(df):
  all_raw_fields=df.schema.fields;
  for i in all_raw_fields:
    if (str(i.dataType)[0:9])=="ArrayType":
      column_list=all_columns_apart_from_the_particular_column(df,i.name);#Select all the columns apart from the array column
      explosion="explode("+i.name+")"+ "as "+i.name;#Explode the array column
      column_list.append(explosion);
      df=df.selectExpr(column_list);
    elif (str(i.dataType)[0:10])=="StructType":
      column_list=all_columns_apart_from_the_particular_column(df,i.name);#Select all the columns apart from the Structure column
      dfstruct=df.schema[i.name].dataType.fields;#Select all the fields of Structure Column
      all_fields_within_structure=expand_structure(dfstruct,i.name);
      proper_name = [ x+" as "+ x.replace('.', "_") for x in all_fields_within_structure ];
      column_list.extend(proper_name)
      df=df.selectExpr(column_list);
  return df;

In [113]:
def encapsulation(df):
  ms=str(df.schema);
  if (("ArrayType" in ms)):#If Arraytype column present , call the flat_df function
    processing=flat_df(df);
    return encapsulation(processing);#Recursive Call
  elif (ms.count("StructType")>1):#If StrctureType column present , call the flat_df function
    processing=flat_df(df);
    return encapsulation(processing);
  else:#Base Case
    return (df);

In [114]:
dff1=encapsulation(df1);
dff2=encapsulation(df2);
dff3=encapsulation(df3);

In [115]:
dff1.show()

+---+
|A_B|
+---+
|  1|
+---+



In [116]:
dff2.show()

+---+---+
|  C|A_B|
+---+---+
|  2|  1|
+---+---+



In [117]:
dff3.show()

+---+---+---+
|  C|A_B|A_D|
+---+---+---+
|  2|  1|  3|
+---+---+---+



https://www.sqlservercentral.com/blogs/schema-evolution-solved-using-delta-lake-databricks

https://kontext.tech/column/spark/381/schema-merging-evolution-with-parquet-in-spark-and-hive

In [118]:
def harmonize_schemas(standard_schema, source_file):
    left_types = {f.name: f.dataType for f in standard_schema}
    right_types = {f.name: f.dataType for f in source_file.schema}
    left_fields = set((f.name, f.dataType,f.nullable) for f in standard_schema)
    right_fields = set((f.name, f.dataType,f.nullable) for f in source_file.schema)
    for l_name, l_type ,l_nullable in left_fields.difference(right_fields):
      if (l_name in right_types):
        r_type=right_types[l_name];
        if (l_type!=r_type):
          source_file = source_file.withColumn(l_name, source_file[l_name].cast(l_type));
        if (l_nullable!=r_nullable):
          source_file.schema[l_name].nullable = l_nullable;
      source_file = source_file.withColumn(l_name, lit(None).cast(l_type));
      source_file.schema[l_name].nullable = l_nullable;
      print("Mismatches with standard_schema :",(l_name,l_type))
    source_file=source_file.select(list(left_types.keys()));
    return source_file;

In [123]:
standard_schema=dff2.schema

In [126]:
correct_df2=harmonize_schemas(standard_schema, dff3)

In [127]:
correct_df2.show()

+---+---+
|  C|A_B|
+---+---+
|  2|  1|
+---+---+



In [129]:
correct_df2.union(dff2).show()

+---+---+
|  C|A_B|
+---+---+
|  2|  1|
|  2|  1|
+---+---+

